In [11]:
import yfinance as yf
import numpy as np
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go

In [12]:
df = yf.download('AAPL', interval='15m')

[*********************100%%**********************]  1 of 1 completed


In [13]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-08-07 11:00:00,213.179993,213.589996,212.649994,213.210007,213.210007,1545176
2024-08-07 11:15:00,213.214996,213.399994,212.770004,213.160004,213.160004,1226174
2024-08-07 11:30:00,213.123993,213.639999,212.479996,212.481903,212.481903,1447642
2024-08-07 11:45:00,212.399994,212.699997,211.960007,212.579895,212.579895,1386171
2024-08-07 12:00:00,212.589996,212.589996,211.909897,211.909897,211.909897,193168


In [14]:
def calculate_sma(data, length:int):
    return ta.sma(data['Close'],length=length)

df['SMA'] = calculate_sma(df, 20)
df.dropna(inplace=True)

In [15]:
df

,Open,High,Low,Close,Adj Close,Volume,SMA
Datetime,,,,,,,
2024-06-10 14:15:00,194.708099,195.169998,192.149994,193.160004,193.160004,7712669,195.336159
2024-06-10 14:30:00,193.149994,194.300003,192.330002,193.669998,193.669998,7148994,195.220659
2024-06-10 14:45:00,193.690002,194.199997,192.899994,193.160004,193.160004,3538071,195.108159
2024-06-10 15:00:00,193.169998,194.169998,192.600601,194.125000,194.125000,3361452,195.055909
2024-06-10 15:15:00,194.130005,194.160004,192.669998,192.964996,192.964996,3097304,194.923159
...,...,...,...,...,...,...,...
2024-08-07 11:00:00,213.179993,213.589996,212.649994,213.210007,213.210007,1545176,209.972311
2024-08-07 11:15:00,213.214996,213.399994,212.770004,213.160004,213.160004,1226174,210.145311
2024-08-07 11:30:00,213.123993,213.639999,212.479996,212.481903,212.481903,1447642,210.309906


In [16]:
def calculate_slope(series,period:int = 5):
    slopes = [0 for _ in range(period-1)]
    for i in range(period-1,len(series)):
        x = np.arange(period)
        y = series[i-period+1:i+1].values
        slope = np.polyfit(x,y,1)[0]
        percent_slope = (slope/y[0]) *100
        slopes.append(percent_slope)
    return slopes

In [17]:
df['Slope'] = calculate_slope(df['SMA'])

In [18]:
df.reset_index(inplace=True,drop=True)

In [19]:
df

,Open,High,Low,Close,Adj Close,Volume,SMA,Slope
0,194.708099,195.169998,192.149994,193.160004,193.160004,7712669,195.336159,0.000000
1,193.149994,194.300003,192.330002,193.669998,193.669998,7148994,195.220659,0.000000
2,193.690002,194.199997,192.899994,193.160004,193.160004,3538071,195.108159,0.000000
3,193.169998,194.169998,192.600601,194.125000,194.125000,3361452,195.055909,0.000000
4,194.130005,194.160004,192.669998,192.964996,192.964996,3097304,194.923159,-0.050720
...,...,...,...,...,...,...,...,...
1015,213.179993,213.589996,212.649994,213.210007,213.210007,1545176,209.972311,0.098099
1016,213.214996,213.399994,212.770004,213.160004,213.160004,1226174,210.145311,0.099791
1017,213.123993,213.639999,212.479996,212.481903,212.481903,1447642,210.309906,0.093405
1018,212.399994,212.699997,211.960007,212.579895,212.579895,1386171,210.489400,0.086281


In [21]:
df['SMA_10'] = calculate_sma(df,10)
df['SMA_20'] = calculate_sma(df,20)
df['SMA_50'] = calculate_sma(df,50)

In [24]:
dfpl = df[:]
fig = go.Figure(data= [go.Candlestick(
    x=dfpl.index,
    open=dfpl['Open'],
    high=dfpl['High'],
    low=dfpl['Low'],
    close=dfpl['Close'],
)])

fig.add_scatter(x=dfpl.index, y=dfpl['SMA_10'], mode='markers', name='SMA 10', marker=dict(color='blue', size=2, ))
fig.add_scatter(x=dfpl.index, y=dfpl['SMA_20'], mode='markers', name='SMA 20', marker=dict(color='red', size=2, ))
fig.add_scatter(x=dfpl.index, y=dfpl['SMA_50'], mode='markers', name='SMA 50', marker=dict(color='green', size=2, ))

fig.show()

In [25]:
def determine_trend(data):
    if data['SMA_10'] > data['SMA_20']  > data['SMA_50']:
        return 2
    elif data['SMA_10'] < data['SMA_20']  < data['SMA_50']:
        return 1
    else:
        return 0

In [26]:
df['Trend'] = df.apply(determine_trend
                        ,axis=1)

In [27]:
df

,Open,High,Low,Close,Adj Close,Volume,SMA,Slope,SMA_10,SMA_20,SMA_50,Trend
0,194.708099,195.169998,192.149994,193.160004,193.160004,7712669,195.336159,0.000000,NaN,NaN,NaN,0
1,193.149994,194.300003,192.330002,193.669998,193.669998,7148994,195.220659,0.000000,NaN,NaN,NaN,0
2,193.690002,194.199997,192.899994,193.160004,193.160004,3538071,195.108159,0.000000,NaN,NaN,NaN,0
3,193.169998,194.169998,192.600601,194.125000,194.125000,3361452,195.055909,0.000000,NaN,NaN,NaN,0
4,194.130005,194.160004,192.669998,192.964996,192.964996,3097304,194.923159,-0.050720,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1015,213.179993,213.589996,212.649994,213.210007,213.210007,1545176,209.972311,0.098099,211.017981,209.972311,208.858065,2
1016,213.214996,213.399994,212.770004,213.160004,213.160004,1226174,210.145311,0.099791,211.483481,210.145311,208.914565,2
1017,213.123993,213.639999,212.479996,212.481903,212.481903,1447642,210.309906,0.093405,211.939671,210.309906,208.931403,2
1018,212.399994,212.699997,211.960007,212.579895,212.579895,1386171,210.489400,0.086281,212.485161,210.489400,208.930701,2
